In [295]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from scipy.stats import skewnorm

import grocery_classes

### Check-out flow

1. Customer approaches the register area
2. Customer queues at the register with the shortest line length
3. **Later feature**: While the customer is at the top of the stack, they can switch to another register with shorter line or fewer items, since now they have had a bit of time to survey the other registers' lines and the contents of their customers' carts
4. Each register processes customers at a rate of n seconds per item, which is a fixed cashier trait
   a. The processing rate can be increased by a constant multiplier if a bagger is added to register
   b. When register is empty (0 items), cashier can "add" a customer from a neighboring queue (FIFO)  
5. The store can open another register if it has the capacity 

In [458]:
# Person: number of items; 
# everyone has perfect vision and 
# the ability to tell which the shortest line is

class Customer:
    
    def __init__(self, name: str, n: int, vessel: str):
        self.name = name
        self.items = n
        if vessel in ["cart", "basket"]:
            self.vessel = vessel
        else:
            raise ValueError("`vessel` must be one of 'cart' or 'basket'")

    def join_register(self, register: Register):
        register.line_length += 1
        register.items += self.items
        register.vessel.append(self.vessel)
        register.customers.append(self.name)

class Cashier:
    def __init__(self, speed: int):
        """A Cashier instance

        Args:
            speed (int): number of items the cashier can process in a minute
        """
        self.speed = speed

class Register:
    
    bagger_multiplier = 1.5

    def __init__(self, index: int, cashier, bagger: bool):
        """A Register instance

        Args:
            index (int): unique identifier
            cashier (Cashier): instance of class Cashier
            bagger (bool): whether register includes a bagger
        """
        self.index = index
        # ! New register always has 0 items, 0 customers in line, and is active
        self.items = 0
        self.line_length = 0
        self.customers = []
        self.vessel = []
        self.active = True
    
        # ! A cashier is required when a register is opened
        if cashier is None:
            self.cashier = None
        else:
            self.cashier = cashier
        # A bagger is optional (you can always add one later with add_bagger())
            self.bagger = bagger
        # If a bagger is added, checkout speed increases
            if bagger:
                self.speed = cashier.speed * self.bagger_multiplier
            else:
                self.speed = cashier.speed

    def activate(self):
        self.active = True

    def deactivate(self):
        self.active = False
    
    def add_bagger(self):
    # TODO: since baggers are just normal employees, add: check Store has enough employees free
        self.bagger = True
        self.speed = self.speed * self.bagger_multiplier

class Store:
    
    def __init__(self, n_registers: int, employees: int):
        
        registers = []
        free_employees = employees

        for i in range(n_registers):
            if free_employees <= 0:
                current_register = Register(index=i, cashier=None, bagger=False)
                current_register.deactivate()
                registers.append(current_register)

            else:
                cashier = Cashier(np.random.normal(30, 5))
                current_register = Register(index=i, cashier = cashier, bagger = False)
                registers.append(current_register)
            
            free_employees -= 1
        self.registers = registers
            

In [475]:
TraderJoes = Store(10, 15)
TraderJoes.registers[0].__dict__
# 

{'index': 0,
 'items': 0,
 'line_length': 0,
 'customers': [],
 'vessel': [],
 'active': True,
 'cashier': <__main__.Cashier at 0x7f2035ea3f40>,
 'bagger': False,
 'speed': 24.273792144256788}

#### Helper Functions

In [345]:
def seq_along(list: list):
    return range(len(list))

def sort_dict(dictionary):
    sorted_d = dict(sorted(dictionary.items()))
    return sorted_d

def sample_skew(skew=4, mean=7, sd=15):
    skewed_dist = skewnorm(skew, mean, sd)
    skew_num = skewed_dist.rvs()
    skew_num_round = round(skew_num, 0)
    if skew_num_round < 1:
        return random.randint(1, 6)
    else: 
        return skew_num_round


if False:
    def find_fewest_items(register_list: list):
        all_items = [reg.items for reg in register_list]
        min_items = [i for i in seq_along(register_list) if register_list[i].items == min(all_items)]
        return min(min_items)

    def find_shortest_line(register_list: list):
        all_line_lengths = [reg.line_length for reg in registers]
        # Find register(s) with the shortest line 
        min_lines = [i for i in seq_along(registers) if registers[i].line_length == min(all_line_lengths)]
        # In case of a tie, arbitrarily pick the register with the lowest index
        # TODO: Break tie with min(items)
        return min(min_lines)



#### Build customers

29.0

##### Build cashiers

In [81]:
cashier = Cashier(random.randint(1, 100))
cashier.__dict__

{'speed': 21}

#### Combine Register Attributes and Weight

In [224]:
def customer_choice():
    """Customer choses register
    Inputs 
        Customer
            - Atrribute Ability vector (i.e., length of attributes)
            - Attribute Preference weights (i.e., length of attributes)
        List of Registers 
            - Each has attribute dict
                - VARIABLE: Line length
                - VARIABLE: # of items
                - CONSTANT: Cashier speed
                - CONSTNAT: Distance from customer
                - CONSTANT: Treates by counter 
    """

# ! ATTRIBUTE MATRIX
my_dict = {"width": 2, "height": 4, "depth": 5}
my_dict = sort_dict(my_dict)
attrs = ('width', 'height')
reg1_attr = [v for k,v in my_dict.items() if k in attrs]
reg1_attr = [2, 60]
register2 = [3,40]
my_matrix = np.matrix([reg1_attr, register2])
# normalize attributes (max)
my_matrix = my_matrix / my_matrix.max(axis=0)

# Add ability vector 
my_matrix += ability
# Apply preference weights
mat_weighted = np.multiply(my_matrix, weights)

# Register-wise sum
summed = np.sum(mat_weighted, axis=1)
sum_array = np.squeeze(np.asarray(summed))
names = ["reg1", "reg2"]
sum_dict = {k:v for k,v in zip(names, sum_array)}

# Pick 
max(sum_dict, key=sum_dict.get)

'reg2'

In [213]:
my_matrix /= my_matrix.max(axis=0)
my_matrix

matrix([[0.93333333, 1.        ],
        [1.        , 0.98412698]])